###Model


In [1]:
!pip install --quiet allennlp
!pip install --quiet allennlp-models
from allennlp_models.pretrained import load_predictor
predictor = load_predictor("structured-prediction-constituency-parser")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 26.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 62.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Output()

Output()

Output()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.3.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
sentence = "The old woman was sitting under a tree and sipping coffee."
preds = predictor.predict(sentence)
print(preds["trees"])

(S (NP (DT The) (JJ old) (NN woman)) (VP (VBD was) (VP (VP (VBG sitting) (PP (IN under) (NP (DT a) (NN tree)))) (CC and) (VP (VBG sipping) (NP (NN coffee))))) (. .))


In [3]:
!python -m spacy download en_core_web_sm

2023-05-04 11:41:02.478178: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.3.0/en_core_web_sm-3.3.0-py3-none-any.whl#egg=en_core_web_sm==3.3.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.5.0
    Uninstalling en-core-web-sm-3.5.0:
      Successfully uninstalled en-core-web-sm-3.5.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [5]:
from nltk.tree import Tree
def get_tree(line):
  test_sentence = line.rstrip('?:!.,;')
  print (test_sentence)
  parser_output = predictor.predict(sentence=test_sentence)
  tree_string = parser_output["trees"]
  tree = Tree.fromstring(tree_string)
  return tree

# test_sentence = "The old woman was sitting under a tree and sipping coffee."
# Tree  = get_tree(test_sentence)
# print(Tree)


S sentence

NP noun phrase

VP verb phrase

PP prepositional phrase

Det determiner

N noun

V verb

P preposition

VBD - Past Tense Verb

JJ - Adjective

etc


In [6]:
# split at right most nounphrase or verbphrase

def get_flattened(t):
    sent_str_final = None
    if t is not None:
        sent_str = [" ".join(x.leaves()) for x in list(t)]
        sent_str_final = [" ".join(sent_str)]
        sent_str_final = sent_str_final[0]
    return sent_str_final

def get_right_most_VP_or_NP(parse_tree,last_NP = None,last_VP = None):
    if len(parse_tree.leaves()) == 1:
        return last_NP,last_VP
    last_subtree = parse_tree[-1]
    if last_subtree.label() == "NP":
        last_NP = last_subtree
    elif last_subtree.label() == "VP":
        last_VP = last_subtree
    
    return get_right_most_VP_or_NP(last_subtree,last_NP,last_VP)


# last_nounphrase, last_verbphrase =  get_right_most_VP_or_NP(Tree)
# last_nounphrase_flattened = get_flattened(last_nounphrase)
# last_verbphrase_flattened = get_flattened(last_verbphrase)

# print ("Original Sentence ",test_sentence)
# print ("last_nounphrase ",last_nounphrase )
# print ("last_verbphrase ",last_verbphrase)
# print ("\n ")
# print ("last_nounphrase ",last_nounphrase_flattened )
# print ("last_verbphrase ",last_verbphrase_flattened)

In [7]:
import re
def get_termination_portion(main_string, sub_string):
    combined_sub_string = sub_string.replace(" ", "")
    main_string_list = main_string.split()
    last_index = len(main_string_list)
    for i in range(last_index):
        check_string_list = main_string_list[i:]
        check_string = "".join(check_string_list)
        check_string = check_string.replace(" ", "")
        if check_string == combined_sub_string:
            return " ".join(main_string_list[:i])

    return None

# longest_phrase_to_use = max(last_nounphrase_flattened, last_verbphrase_flattened,key = len)
# print ("Ending phrase: ", longest_phrase_to_use)

# longest_phrase_to_use = re.sub(r"-LRB- ", "(", longest_phrase_to_use)
# longest_phrase_to_use = re.sub(r" -RRB-", ")", longest_phrase_to_use)
# split_sentence = get_termination_portion(test_sentence, longest_phrase_to_use)
# print ("Original sentence : ",test_sentence)
# print ("Original sentence after splitting at ending phrase: ",split_sentence)

###Para

In [8]:
!pip install --quiet flashtext==2.7
!pip install git+https://github.com/boudinfl/pke.git

  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-ghgh760c
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-ghgh760c
  Resolved https://github.com/boudinfl/pke.git to commit ebd6e5754b4156a61a4ec6c4c283e821d11a36be
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 16.7 MB/s eta 0:00:00
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160677 sha256=e43b3923fc44341ae01bf181ee1f1736700a29afa8a8fa4054fefb1b242d07b0
  Stored in directory: /tmp/pip-ephem-wheel-cache-o8vaqjay/wheels/8c/07/29/6b35bed2aa36e33d77ff3677eb716965ece4d2e56639ad0aab
Successfully built pke


In [9]:
!pip install scipy==1.8.0
!pip install networkx==2.6
!pip install --quiet gradio==3.14.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.0 MB/s eta 0:00:00
Reason for being yanked: Need to resolve: https://github.com/networkx/networkx/pull/4967
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 819.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0

In [10]:
!python -m spacy info

2023-05-04 11:42:38.394572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.3.2                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-5.10.147+-x86_64-with-glibc2.31
Python version   3.10.11                       
Pipelines        en_core_web_sm (3.3.0)        



In [11]:
import json
import requests
import string
import re
import nltk
import string
import itertools
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import pke
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import traceback
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# text = """Grapes are round and small in shape. And it is one of the delicious and refreshing fruit.

# Grapes are sour and sweet.

# It is a lush fruit, contains a lot of juice.

# Grapes are used to making many things and drinks like juices, jams, wines, etc.

# Grapes are mostly used in every part of the world. And its byproducts are more famous than the fruit.

# Grapes are grown in bunches, and it requires a hot and dry climate to grow because humidity may spread diseases.

# Grapes are of three colors green, black and red. But the most loved one is green as it is more nutritious, and the other two color variants are more used for preparing its byproducts.

# The largest grape producer in the world is Italy and France.

# In India, grapes are grown mostly in Maharashtra, Karnataka, Andhra Pradesh, and Tamil Nadu. It is grown in the Northern part of India as a seasonal crop.

# Almost 10,000 types of grapes are found in the world, and more than 20 of them are grown in India.

# It contains a lot of fiber, minerals, and vitamins, which make it nutritious.

# Generally, Vitamin A and C are found in grapes. It is a citrus fruit.”"""

In [13]:
def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

# text = """There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is pulled into the mantle, forming magma that erupts as volcanoes. When a line of volcanoes forms along a subduction zone, they make up a volcanic arc. The edges of the Pacific plate are long subduction zones lined with volcanoes. This is why the Pacific rim is called the “Pacific Ring of Fire.”"""

# sentences = tokenize_sentences(text)
# print (sentences)

In [14]:
def get_keywords(text):
    out=[]
    try:
        extractor = pke.unsupervised.YAKE()
        extractor.load_document(input=text,language='en')
        grammar = r"""
                NP:
                    {<NOUN|PROPN>+}
            """
        extractor.ngram_selection(n=1)
        extractor.grammar_selection(grammar=grammar)
        extractor.candidate_selection(n=1)

        extractor.candidate_weighting(window=3,
                                      use_stems=False)

        keyphrases = extractor.get_n_best(n=30)
        

        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out

# keywords = get_keywords(text)[:8]
# print ("keywords: ",keywords)

In [15]:
from pprint import pprint
def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

# keyword_sentence_mapping_noun_verbs_adj = get_sentences_for_keyword(noun_verbs_adj, sentences)
# pprint (keyword_sentence_mapping_noun_verbs_adj)

In [16]:
# sentences = []
# for key in keyword_sentence_mapping_noun_verbs_adj:
#         if len(keyword_sentence_mapping_noun_verbs_adj[key])>0:
#             sent = keyword_sentence_mapping_noun_verbs_adj[key][0]
#             sentences.append(sent)
#             # print(key, sent)
# print(sentences)

In [17]:
def get_sentences(text):
  sentences = tokenize_sentences(text)
  # noun_verbs_adj = get_noun_adj_verb(text)
  noun_verbs_adj = get_keywords(text)
  keyword_sentence_mapping_noun_verbs_adj = get_sentences_for_keyword(noun_verbs_adj, sentences)
  sentences = []
  for key in keyword_sentence_mapping_noun_verbs_adj:    
    if len(keyword_sentence_mapping_noun_verbs_adj[key])>0:
      sent = keyword_sentence_mapping_noun_verbs_adj[key][0]
      sentences.append(sent)
      # print(key, sent)
  return sentences




In [18]:
import re

def splits_sentences(unique_sentences):
  split_sentences = []
  for sentence in unique_sentences[:5]:
    tree = get_tree(sentence)
    sentence = sentence.rstrip('?:!.,;')
    last_nounphrase, last_verbphrase =  get_right_most_VP_or_NP(tree)
    last_nounphrase_flattened = get_flattened(last_nounphrase)
    last_verbphrase_flattened = get_flattened(last_verbphrase)
    # print(len(sentence))
    # print ("last_nounphrase ",last_nounphrase_flattened )
    # print ("last_verbphrase ",last_verbphrase_flattened)
    # longest_phrase_to_use = max(last_nounphrase_flattened, last_verbphrase_flattened,key = len)
    if last_nounphrase_flattened is None or last_verbphrase_flattened is None:
      # print("F1: ", sentence)
      split_sentences.append(sentence)
    else:
      longest_phrase_to_use =  last_nounphrase_flattened
      # print ("Ending phrase: ", longest_phrase_to_use)
      longest_phrase_to_use = re.sub(r"-LRB- ", "(", longest_phrase_to_use)
      longest_phrase_to_use = re.sub(r" -RRB-", ")", longest_phrase_to_use)
      split_sentence = get_termination_portion(sentence, longest_phrase_to_use)
      # print ("Original sentence after splitting at ending phrase: ",split_sentence)
      split_sentences.append(split_sentence)
      print("/n")
  return split_sentences 

In [19]:
# split_Sentences = false_sentences(unique_sentences)
# print(split_Sentences)

###false sentences

In [20]:
# !pip install --upgrade pip
# !pip install transformers
# !pip install sacremoses==0.0.53
!pip install torch transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
# import tensorflow as tf
# from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
# GPT2tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# print(GPT2tokenizer.eos_token_id)
# GPT2model = TFGPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=GPT2tokenizer.eos_token_id)
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2',pad_token_id=tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [22]:
import nltk
nltk.download('punkt')
from nltk import tokenize

def false_sentence(partial_sentence):
  # input_ids = GPT2tokenizer.encode(partial_sentence,return_tensors='tf')
  input_ids = tokenizer.encode(partial_sentence, return_tensors='pt')
  maximum_length = len(partial_sentence.split())+40
  sample_outputs = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=maximum_length, 
    top_p=0.90, # 0.85 
    top_k=30,   #30
    repetition_penalty  = 10.0,
    num_return_sequences=10)
  generated_sentences=[]
  for i, sample_output in enumerate(sample_outputs):
    decoded_sentence = tokenizer.decode(sample_output, skip_special_tokens=True)
    final_sentence = tokenize.sent_tokenize(decoded_sentence)[0]
    generated_sentences.append(final_sentence)
  return generated_sentences

# generated_sentences = false_sentence("The old woman was sitting under a tree and")
# print(generated_sentences);

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


###true sentences

In [23]:
! pip install git+https://github.com/PrithivirajDamodaran/Parrot.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PrithivirajDamodaran/Parrot.git to /tmp/pip-req-build-y7g9ewoo
  Running command git clone --filter=blob:none --quiet https://github.com/PrithivirajDamodaran/Parrot.git /tmp/pip-req-build-y7g9ewoo
  Resolved https://github.com/PrithivirajDamodaran/Parrot.git to commit 720a87a1ee557d8ed8d9a021adbdd1dd5616c5f9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.5 MB/s eta 0:00:00
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8608 sha256=d2ff280a7660bfa45cb632e4b40702b5a5083f475f0fca327a0040b9dd415d79
  Stored in directory: /tmp/pip-ephem-wheel-cache-pdpx_f13/wheels/0f/3e/1a/2183ef66216007ff3

In [24]:
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

In [25]:
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

In [26]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [42]:
def true_sentence(unique_sentences):
  true_sentences = []
  for sentence in unique_sentences[5:8]:
    print("-"*100)
    print("Input_phrase: ", sentence)
    print("-"*100)
    para_phrases = parrot.augment(input_phrase=sentence)
    # for para_phrase in para_phrases:
    if para_phrases is not None:
      print(para_phrases[0][0])
      true_sentences.append(para_phrases[0][0])
  return true_sentences

In [28]:
# for sentence in true_sentences:
#   print("T: ", sentence)

###sort

In [29]:
!pip install sentence-transformers==1.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.0-py3-none-any.whl size=114162 sha256=223e5eda73d6d86cd9977d66a1fa9feb41628e81679bc2ed6cbaec8bbd58e7ea
  Stored in directory: /root/.cache/pip/wheels/e3/cf/22/39c6ebe4d9aaa437046fd04dfc9b55bd68d57ede0382e76e23
Successfully built sentence-transformers
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [30]:
from sentence_transformers import SentenceTransformer, util
BERT_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/555 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/505 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [31]:
original_sentence = "The old woman was sitting under a tree and sipping coffee."

In [32]:
import scipy

def sort_Sentences(generated_sentences,original_sentence):
  false_sentences_embeddings = BERT_model.encode(generated_sentences)
  original_sentence_embedding = BERT_model.encode([original_sentence])
  distances = scipy.spatial.distance.cdist(original_sentence_embedding, false_sentences_embeddings, "cosine")[0]
  results = zip(range(len(distances)), distances)
  results = sorted(results, key=lambda x: x[1])
  dissimilar_sentences =[]
  for idx, distance in results:
    dissimilar_sentences.append(generated_sentences[idx])
  return dissimilar_sentences

# dissimilar_sentences=sort_Sentences(generated_sentences,original_sentence);
# print(dissimilar_sentences)

###output


In [33]:
import re
def generate_sentences(split_sentences):
  false= []
  for sentence in split_sentences[:5]:
      generated_Sentences = false_sentence(sentence)
      dissimilar_sentences=sort_Sentences(generated_Sentences,sentence);
      false_sentences_list_final = reversed(dissimilar_sentences)
      false_sentence_list = list(false_sentences_list_final)
      false.append(false_sentence_list[0])
      #print("F: ", false_sentence_list[0])
   
  return false

In [46]:
# text = """There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is pulled into the mantle, forming magma that erupts as volcanoes. When a line of volcanoes forms along a subduction zone, they make up a volcanic arc. The edges of the Pacific plate are long subduction zones lined with volcanoes. This is why the Pacific rim is called the “Pacific Ring of Fire.”"""

# text = """The Battle of Panipat was a significant military confrontation that took place in northern India on January 14, 1761. It was fought between the Maratha Empire and the invading forces of Ahmad Shah Durrani, who was leading the Afghan Empire. The battle is considered one of the largest and bloodiest in the 18th century and resulted in a decisive victory for Ahmad Shah Durrani.

# The Maratha army was led by Sadashivrao Bhau, who was a nephew of the Peshwa Baji Rao I. The Marathas had a massive army that outnumbered the Afghans, but they were hampered by poor planning and infighting. In contrast, the Afghan forces were highly disciplined and well-organized, and they were able to exploit the weaknesses of the Maratha army.

# The battle lasted for several hours and resulted in the deaths of tens of thousands of soldiers. The Maratha army suffered a crushing defeat, and many of their leaders were killed, including Sadashivrao Bhau. The aftermath of the battle led to significant changes in the political landscape of India, as the Maratha Empire began to decline, and the Afghan Empire became more influential in the region."""


text = """The Mughal Empire was a powerful empire that ruled much of India from the 16th to the 19th century. It was founded by Babur, a descendant of Genghis Khan and Timur, who established the dynasty in 1526. Under the rule of Akbar, the empire reached its peak, with territories extending from Afghanistan to Bengal.

The Mughal Empire was renowned for its cultural achievements, including its architecture, art, and literature. The Taj Mahal, one of the most famous buildings in the world, was built during this time. The empire was also known for its religious tolerance, as Akbar promoted a policy of unity among different faiths.

However, the decline of the Mughal Empire began in the late 17th century, with weak rulers and increasing power struggles among the nobles. The empire eventually fell to the British in the mid-19th century, marking the end of the Mughal era in India. Despite its eventual downfall, the Mughal Empire left a lasting impact on Indian history and culture."""

sentences = get_sentences(text)
# print(sentences)
unique_sentences = list(set(sentences))
print(unique_sentences)
import random
random.shuffle(unique_sentences)
print(unique_sentences)
# print(len(unique_sentences))
split_Sentences = splits_sentences(unique_sentences)
print(split_Sentences)


['The Mughal Empire was a powerful empire that ruled much of India from the 16th to the 19th century.', 'Under the rule of Akbar, the empire reached its peak, with territories extending from Afghanistan to Bengal.', 'The Mughal Empire was renowned for its cultural achievements, including its architecture, art, and literature.', 'It was founded by Babur, a descendant of Genghis Khan and Timur, who established the dynasty in 1526.', 'However, the decline of the Mughal Empire began in the late 17th century, with weak rulers and increasing power struggles among the nobles.', 'The empire was also known for its religious tolerance, as Akbar promoted a policy of unity among different faiths.', 'The empire eventually fell to the British in the mid-19th century, marking the end of the Mughal era in India.', 'The Taj Mahal, one of the most famous buildings in the world, was built during this time.']
['The empire eventually fell to the British in the mid-19th century, marking the end of the Mugha

In [47]:
false_generated_sentences = generate_sentences(split_Sentences)
print(false_generated_sentences)

['The empire eventually fell to the British in the mid-19th century, marking the end of the Mughal era in India and a time when many Hindu religious communities were not permitted at all.', 'It was founded by Babur, a descendant of Genghis Khan and Timur, who established the dynasty in 1377.', 'The Mughal Empire was a powerful empire that ruled much of India from the 16th to 20-20 AD.', 'However, the decline of the Mughal Empire began in the late 17th century, with weak rulers and increasing power struggles among Indian tribes.', 'The Taj Mahal, one of the most famous buildings in the world, was built during a time when Muslims lived very modestly and had little food for their children.']


In [48]:
true_generated_sentences = true_sentence(unique_sentences)

----------------------------------------------------------------------------------------------------
Input_phrase:  The Mughal Empire was renowned for its cultural achievements, including its architecture, art, and literature.
----------------------------------------------------------------------------------------------------
The Mughal Empire was renowned for its cultural achievements, including its architecture, art, and literature.
----------------------------------------------------------------------------------------------------
Input_phrase:  The empire was also known for its religious tolerance, as Akbar promoted a policy of unity among different faiths.
----------------------------------------------------------------------------------------------------
the empire was also known for its religious tolerance as akbar promoted a policy of unity between different religions
----------------------------------------------------------------------------------------------------
Input_phra

In [49]:
for sentence in true_generated_sentences:
    false_generated_sentences.append(sentence)
print(false_generated_sentences)

['The empire eventually fell to the British in the mid-19th century, marking the end of the Mughal era in India and a time when many Hindu religious communities were not permitted at all.', 'It was founded by Babur, a descendant of Genghis Khan and Timur, who established the dynasty in 1377.', 'The Mughal Empire was a powerful empire that ruled much of India from the 16th to 20-20 AD.', 'However, the decline of the Mughal Empire began in the late 17th century, with weak rulers and increasing power struggles among Indian tribes.', 'The Taj Mahal, one of the most famous buildings in the world, was built during a time when Muslims lived very modestly and had little food for their children.', 'The Mughal Empire was renowned for its cultural achievements, including its architecture, art, and literature.', 'the empire was also known for its religious tolerance as akbar promoted a policy of unity between different religions', 'the empire reached its peak under the rule of akbar with territori

In [50]:
import random
random.shuffle(false_generated_sentences)
print("Original Text: ")
print(text)
print("\n")
print("True or False:- ")
for sentence in false_generated_sentences:
  print(sentence)

Original Text: 
The Mughal Empire was a powerful empire that ruled much of India from the 16th to the 19th century. It was founded by Babur, a descendant of Genghis Khan and Timur, who established the dynasty in 1526. Under the rule of Akbar, the empire reached its peak, with territories extending from Afghanistan to Bengal.

The Mughal Empire was renowned for its cultural achievements, including its architecture, art, and literature. The Taj Mahal, one of the most famous buildings in the world, was built during this time. The empire was also known for its religious tolerance, as Akbar promoted a policy of unity among different faiths.

However, the decline of the Mughal Empire began in the late 17th century, with weak rulers and increasing power struggles among the nobles. The empire eventually fell to the British in the mid-19th century, marking the end of the Mughal era in India. Despite its eventual downfall, the Mughal Empire left a lasting impact on Indian history and culture.


